<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Fundamentos de Aprendizado de Máquina
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)
<br>
Aluno [Anderson Júnior](https://www.github.com/ajrsz/)
---

# **Tópicos**

<ol type="1">
  <li>Teoria;</li>
  <li>Atributos categóricos;</li>
  <li>Atributos numéricos;</li>
  <li>Dados faltantes.</li>
</ol>

---

# **Exercícios**

## 1\. Pinguins 

Neste exercício, vamos utilizar uma base de dados com informações sobre penguins. A idéia é preparar a base de dados para prever a espécie do penguin (variável resposta) baseado em suas características físicas e geográficas (variáveis preditivas).

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns

In [12]:
data = sns.load_dataset('penguins')

In [23]:
data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


### **1.1. Valores nulos** 

A base de dados possui valores faltantes, utilize os conceitos da aula para trata-los.

In [24]:
#Decidi só apagar os dados nulos
df_limpo = data.dropna()
print(df_limpo.head())


  species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
4  Adelie  Torgersen            36.7           19.3              193.0   
5  Adelie  Torgersen            39.3           20.6              190.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
4       3450.0  Female  
5       3650.0    Male  


### **1.2. Variáveis numéricas** 

Identifique as variáveis numéricas e crie uma nova coluna **padronizando** seus valores. A nova coluna deve ter o mesmo nome da coluna original acrescidade de "*_std*".

> **Nota**: Você não deve tratar a variável resposta.

In [25]:
# Primeiro tentei identificar as variáveis numéricas
numerical_cols = df_limpo.select_dtypes(include=['float64', 'int64']).columns

# Fui padronizar com uma função for e tive um problema com 'Setting With Copy Warning', aí fui descobrir o assign
for col in numerical_cols:
    mean = df_limpo[col].mean()
    std = df_limpo[col].std()
    df_limpo = df_limpo.assign(**{col + '_std': (df_limpo[col] - mean) / std})

# Limpeza das colunas
df_limpo = df_limpo.drop(columns=[col for col in numerical_cols if col != 'species'])

print(df_limpo.head())

  species     island     sex  bill_length_mm_std  bill_depth_mm_std  \
0  Adelie  Torgersen    Male           -0.894695           0.779559   
1  Adelie  Torgersen  Female           -0.821552           0.119404   
2  Adelie  Torgersen  Female           -0.675264           0.424091   
4  Adelie  Torgersen  Female           -1.333559           1.084246   
5  Adelie  Torgersen    Male           -0.858123           1.744400   

   flipper_length_mm_std  body_mass_g_std  
0              -1.424608        -0.567621  
1              -1.067867        -0.505525  
2              -0.425733        -1.188572  
4              -0.568429        -0.940192  
5              -0.782474        -0.691811  


### **1.3. Variáveis categóricas** 

Identifique as variáveis categóricas nominais e ordinais, crie uma nova coluna aplicando a técnica correta de conversão a seus valores. A nova coluna deve ter o mesmo nome da coluna original acrescidade de "*_nom*" ou "*_ord*".

> **Nota**: Você não deve tratar a variável resposta.

In [26]:
# Identificar as variáveis categóricas(embora só tenha nominal), excluindo a variável resposta 'species'
categorical_cols = df_limpo.select_dtypes(include=['object']).columns.drop('species')

# Aplicar a técnica de One-Hot Encoding utilizando o 'dummies' junto da limpeza
for col in categorical_cols:
    dummies = pd.get_dummies(df_limpo[col], prefix=col + '_nom')
    df_limpo = pd.concat([df_limpo, dummies], axis=1)
    df_limpo.drop(columns=[col], inplace=True)

# Conversão dos valores booleanos em 0's e 1's
df_limpo = df_limpo.astype({col: int for col in df_limpo.columns if df_limpo[col].dtype == 'bool'})

print(df_limpo.head())

  species  bill_length_mm_std  bill_depth_mm_std  flipper_length_mm_std  \
0  Adelie           -0.894695           0.779559              -1.424608   
1  Adelie           -0.821552           0.119404              -1.067867   
2  Adelie           -0.675264           0.424091              -0.425733   
4  Adelie           -1.333559           1.084246              -0.568429   
5  Adelie           -0.858123           1.744400              -0.782474   

   body_mass_g_std  island_nom_Biscoe  island_nom_Dream  island_nom_Torgersen  \
0        -0.567621                  0                 0                     1   
1        -0.505525                  0                 0                     1   
2        -1.188572                  0                 0                     1   
4        -0.940192                  0                 0                     1   
5        -0.691811                  0                 0                     1   

   sex_nom_Female  sex_nom_Male  
0               0           

### **1.4. Limpeza** 

Descarte as colunas originais e mantenha apenas a variável resposta e as variáveis preditivas com o sufixo *_std*", *_nom*" e "*_ord*". 

In [ ]:
# Já realizado

---